<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/main/Fall_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and frameworks

In [1]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200


Get the sisfall and sisfall_enhanced dataset

In [ ]:
!gdown --id 1kyTRhIFhqwRkf9gERof1Xm5FVQ-klLVA
!gdown --id 1gvOuxPc8dNgTnxuvPcVuCKifOf98-TV0
!unzip SisFall_dataset.zip
!unzip SisFall_enhanced.zip

# Implementation

Get all addresses

In [3]:
def get_file_name(path):
  allfiles = []
  allFolders = glob.glob(path + "*")
  for files in allFolders:
      allfiles.append(glob.glob(files+"/*.txt"))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')
  return np.hstack(allfiles)

In [4]:
get_file_name("SisFall_dataset/")

array(['SisFall_dataset/SE02/D11_SE02_R01.txt',
       'SisFall_dataset/SE02/D12_SE02_R04.txt',
       'SisFall_dataset/SE02/D08_SE02_R03.txt', ...,
       'SisFall_dataset/SE08/D10_SE08_R01.txt',
       'SisFall_dataset/SE08/D14_SE08_R02.txt',
       'SisFall_dataset/SE08/D15_SE08_R01.txt'], dtype='<U37')

read dataset from address path

In [5]:
def read_data(data_path):
    data = pd.read_csv(data_path, header=None)
    data.columns = ['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z', 'MMA8451Q_x',
                    'MMA8451Q_y', 'MMA8451Q_z']
    data['MMA8451Q_z'] = data['MMA8451Q_z'].map(lambda x: str(x)[:-1])
    for name in data.columns :
      data[name] = data[name].astype(float)
    return data

Feature Extraction

In [6]:
def add_features(dataset,data_path):
    new_dataset = pd.DataFrame()
    new_dataset['acc_1'] = dataset.apply(
        lambda row: sqrt((row.ADXL345_x ** 2 + row.ADXL345_y ** 2 + row.ADXL345_z ** 2)), axis=1)
    new_dataset['acc_2'] = dataset.apply(
        lambda row: sqrt((row.MMA8451Q_x ** 2 + row.MMA8451Q_y ** 2 + row.MMA8451Q_z ** 2)), axis=1)
    new_dataset['geo'] = dataset.apply(
        lambda row: sqrt((row.ITG3200_x ** 2 + row.ITG3200_y ** 2 + row.ITG3200_z ** 2)), axis=1)
    new_dataset['label'] = get_label(data_path)
    return np.round(new_dataset.to_numpy(),2)


Get the Label

In [7]:
def get_label(data_path):
    label = data_path[21]
    if label =='D':
      return int(0)
    elif label =='F':  
      label_path = data_path.replace('dataset','enhanced')
      labels = pd.read_csv(label_path,header=None)
      labels[labels == 2] = 1
      return labels


Split Dataset to train and test

In [8]:
def split_address(dataset_address):
  np.random.shuffle(dataset_address)
  train, test = np.split(dataset_address, [int(len(dataset_address)*0.7)])
  return train, test

Extract features from All addresses

In [9]:
def datasets_to_nparray(datasets_address_array):
  result = np.empty((0, 4), int)
  for address in datasets_address_array:
    result = np.concatenate(
        (result,add_features(read_data(address),address)),axis=0)
  return result

Windowing of the dataset

In [10]:
def windowing(dataset,window_size):
  window = window_size * (dataset.shape[1]-1)
  cut = dataset.shape[0] % window_size
  feature = dataset[:-cut,0:-1]
  label = dataset[:-cut,-1]
  feature = feature.ravel().reshape(feature.size//window,window)
  label = label.reshape(label.size//window_size,window_size)
  label = label.sum(axis=1)
  label[label > 0] = 1
  return feature,label

Save the Train and Test Dataset as Tensor

In [11]:
def dataset_to_tensor(test,train,window_size):
  test_feature , test_label = windowing(datasets_to_nparray(test),window_size)
  np.savez('Sisfall_data_test', inputs=test_feature, targets=test_label)
  train_feature , train_label = windowing(datasets_to_nparray(train),window_size)
  np.savez('Sisfall_data_train', inputs=train_feature, targets=train_label)


# Main

Data Pre-processing

In [12]:
train, test = split_address(get_file_name("SisFall_dataset/"))

In [13]:
dataset_to_tensor(test,train,window_size)

In [14]:
npz = np.load("Sisfall_data_train.npz")
train_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
train_targets = npz["targets"].astype(np.int)


npz = np.load("Sisfall_data_test.npz")
test_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
test_targets = npz["targets"].astype(np.int)

In [15]:
print(train_inputs.shape)
print(train_targets.shape)
print(test_inputs.shape)
print(test_targets.shape)

(54543, 600)
(54543,)
(24751, 600)
(24751,)


Neural Network Training

In [16]:
input_size = 600
output_size = 1
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(input_size,activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 50

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          shuffle = True,
          verbose = 1
          )

Epoch 1/50
546/546 [==============================] - 5s 8ms/step - loss: 0.1066 - accuracy: 0.9674
Epoch 2/50
546/546 [==============================] - 4s 8ms/step - loss: 0.0724 - accuracy: 0.9762
Epoch 3/50
546/546 [==============================] - 4s 8ms/step - loss: 0.0642 - accuracy: 0.9780
Epoch 4/50
546/546 [==============================] - 4s 8ms/step - loss: 0.0583 - accuracy: 0.9803
Epoch 5/50
546/546 [==============================] - 9s 17ms/step - loss: 0.0554 - accuracy: 0.9813
Epoch 6/50
546/546 [==============================] - 10s 18ms/step - loss: 0.0525 - accuracy: 0.9821
Epoch 7/50
546/546 [==============================] - 8s 15ms/step - loss: 0.0491 - accuracy: 0.9831
Epoch 8/50
546/546 [==============================] - 10s 19ms/step - loss: 0.0469 - accuracy: 0.9842
Epoch 9/50
546/546 [==============================] - 12s 22ms/step - loss: 0.0460 - accuracy: 0.9846
Epoch 10/50
546/546 [==============================] - 12s 22ms/step - loss: 0.0447 - accura

Neural Network Evaluation

In [17]:
loss, accuracy = model.evaluate(test_inputs, test_targets, verbose=1)

774/774 [==============================] - 7s 8ms/step - loss: 0.1184 - accuracy: 0.9801


Confusion Matrix

In [26]:
predictions = model.predict(test_inputs)
conf_matrix = tf.math.confusion_matrix(labels=test_targets,
                                       predictions=predictions).numpy()
print("------ Confusion Matrix Created ------")
print()
print(conf_matrix)
print()

precision = conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[0][1]).round()
recall = conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0]).round()
f1Score = (2 * precision * recall)/(precision + recall).round()
statics = pd.DataFrame([[precision,recall,f1Score]],columns=["Precision","Recall","F1score"])
statics

------ Confusion Matrix Created ------

[[23673     7]
 [  738   333]]



,Precision,Recall,F1score
0,0.979412,0.310924,0.609046


### SVM and Logistic Regression

In [19]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
x = train_inputs
y = train_targets
x_test = test_inputs
y_test = test_targets

Logistic Regression

In [20]:
reg = LogisticRegression()
reg.fit(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Logistic Regression Evaluation

In [21]:
print("score on test: " + str(reg.score(x_test, y_test)))
print("score on train: "+ str(reg.score(x, y)))
print()
y_pred = reg.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.9571734475374732
score on train: 0.9592981684175788

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     23680
           1       0.52      0.17      0.26      1071

    accuracy                           0.96     24751
   macro avg       0.74      0.58      0.62     24751
weighted avg       0.94      0.96      0.95     24751

[[23506   174]
 [  886   185]]


Svm Model


In [22]:
svm=LinearSVC(C = 0.0001)
svm.fit(x, y)


LinearSVC(C=0.0001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Svm Model Evaluation

In [23]:
print("score on test: "  + str(svm.score(x_test, y_test)))
print("score on train: " + str(svm.score(x, y)))
print()
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.9569714354975557
score on train: 0.9531378912051043

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     23680
           1       0.59      0.02      0.03      1071

    accuracy                           0.96     24751
   macro avg       0.78      0.51      0.51     24751
weighted avg       0.94      0.96      0.94     24751

[[23667    13]
 [ 1052    19]]
